# Strategy walkthrough

This notebook demos how to hit the FastAPI quant backtest, explore parameters, and inspect the resulting equity curve. The code uses the local `run_quant_backtest` engine for reproducibility and illustrates the SMA/RSI strategy discussed in the docs.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from backend.app import quant_engine
from backend.app.models import QuantBacktestRequest, QuantStrategyConfig

spy_df = pd.read_csv('../sample_data/spy_sample.csv', parse_dates=['date']).set_index('date')

def mock_fetch_price_history(symbols, start, end, field='Close'):
    series = spy_df[field.lower()]
    return pd.DataFrame({symbols[0]: series})

quant_engine.fetch_price_history = mock_fetch_price_history


In [ ]:
def run_strategy(fast, slow):
    config = QuantStrategyConfig(
        symbol='SPY',
        timeframe='1D',
        start_date='2025-01-02',
        end_date='2025-01-15',
        initial_capital=100000,
        position_mode='long_flat',
        sma_fast=fast,
        sma_slow=slow,
        rsi_period=14,
        rsi_overbought=70,
        rsi_oversold=30,
        use_sma=True,
        use_rsi=True,
    )
    request = QuantBacktestRequest(
        strategy=config,
        slippage_bps=0.5,
        commission_per_trade=0,
        max_position_size=1.0,
    )
    result = quant_engine.run_quant_backtest(request)
    returns = np.array(result['returns'])
    sharpe = (np.mean(returns) * 252) / (np.std(returns) * np.sqrt(252) + 1e-8)
    return sharpe, result

grid = [(fast, slow) for fast in [5, 10, 20] for slow in [30, 50, 100]]
sweep = { }
for fast, slow in grid:
    sharpe, _ = run_strategy(fast, slow)
    sweep[(fast, slow)] = sharpe

heatmap = pd.DataFrame(
    [[sweep[(fast, slow)] for slow in [30, 50, 100]] for fast in [5, 10, 20]],
    index=[5, 10, 20],
    columns=[30, 50, 100],
)

plt.imshow(heatmap, aspect='auto', cmap='YlGnBu')
plt.colorbar(label='Sharpe')
plt.xlabel('SMA slow')
plt.ylabel('SMA fast')
plt.title('SMA grid: Sharpe')
plt.xticks(range(len(heatmap.columns)), heatmap.columns)
plt.yticks(range(len(heatmap.index)), heatmap.index)
plt.show()


## Investigating a corner case
Execute the `run_strategy` helper with `fast=10, slow=200` to inspect the equity curve, trades, and metrics. The API-friendly object returned by `run_quant_backtest` can be serialized to JSON and stored in `QuantLabStore`.